In [29]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize
import time
from math import comb

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.

CHECK DELIST:
- Purpose: checks that there is actually valid market data for this stock
- Code: if 'Close' not in data.columns or data['Close'].isnull().all():

PORTFOLIO GENERATOR
- Purpose: calculate how many shares of each stock to buy, and how much to pay in fees
- Code:

portfolio_value = 1000000 * weights  # Portfolio value based on optimized weights

shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers])  # Number of shares

fee = np.minimum(3.95, 0.001 * shares)

portfolio_val -= fee


In [30]:
#October 01, 2023 to September 30, 2024, 2023

START_DATE = '2023-10-01'
END_DATE = '2024-11-20'

 # date on which we will take stock Close prices
TARGET_DATE = pd.to_datetime('2024-10-22').date()

INVESTMENT = 1000000

CSV_FILE = "Tickers.csv"

In [31]:
# read in CSV
csv = pd.read_csv(CSV_FILE, header=None)

display(csv)

,0
0,asdfaasdf.to
1,AAPL
2,ABBV
3,LOW
4,AUST
5,HOOD
6,INVALIDTIC
7,AMZN
8,AXP
9,BAC


In [32]:
# filter out invalid stocks
def validity(tickers):

    # list to keep all of the valid stocks
    final_list = []

    for ticker in tickers:
        #only append the stock to the final stock list if it is isn't delisted, has a US or CAD currency, and has the required monthly share volume
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker) and
            ticker not in final_list):
                final_list.append(ticker)
    return final_list

# checks if the consumed Ticker is delisted
def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        time.sleep(0.3)
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock
            if 'Close' not in data.columns or data['Close'].isnull().all():
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

# checks if the consumed Ticker meets the requirement of an average monthly share volume of 100,000 shares
def check_volume(ticker):
    volume = yf.Ticker(ticker).history(start=START_DATE, end='2024-09-30')['Volume']
    time.sleep(0.3)
    volume.index = pd.to_datetime(volume.index)
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

# checks whether or not the consumed Ticker is listed in CAD or in USD
def check_currency(ticker):
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # if there's an error retrieving the data, we can assume it doesnt satisfy the required monthly volume

# create a new DataFrame only containing the valid Tickers in the consumed CSV
tickers_raw = validity(csv[0])
tickers_raw

$ASDFAASDF.TO: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$INVALIDTIC: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$HDFC.NS: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AW.TO: possibly delisted; no price data found  (1d 2023-10-01 -> 2024-09-30) (Yahoo error = "Data doesn't exist for startDate = 1696132800, endDate = 1727668800")


['AAPL',
 'ABBV',
 'LOW',
 'HOOD',
 'AMZN',
 'AXP',
 'BAC',
 'BK',
 'SQ',
 'VZ',
 'CMCSA',
 'SHOP',
 'COST',
 'CSCO',
 'CVS',
 'GM',
 'GOOG',
 'JPM',
 'IBM',
 'ORCL',
 'OXY',
 'DUOL',
 'PEP',
 'T.TO',
 'SLB',
 'SO',
 'SPG',
 'SHOP.TO',
 'RY.TO',
 'TD.TO']

In [33]:
def days_to_drop(markets):

    markets.dropna(inplace=True)
    months = markets.resample('MS').bfill().index
    
    dates = []

    for month in months:
        temp = markets[(markets.index.month==month.month) & (markets.index.year==month.year)].index
        if len(temp) < 18:
            dates.extend(temp)
    
    return dates

In [34]:
tickers_raw_df = pd.DataFrame()

# calculate and store all returns for all all Tickers
for ticker in tickers_raw:
    data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close'].pct_change()
    data.drop(index=data.index[0], inplace=True)
    time.sleep(0.3)
    tickers_raw_df[ticker] = data  # this code calculates daily returns

tickers_raw_df.index = pd.to_datetime(tickers_raw_df.index.strftime("%Y-%m-%d"))

tickers_raw_df

,AAPL,ABBV,LOW,HOOD,AMZN,AXP,BAC,BK,SQ,VZ,...,OXY,DUOL,PEP,T.TO,SLB,SO,SPG,SHOP.TO,RY.TO,TD.TO
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03,-0.007770,-0.005531,-0.027122,-0.020555,-0.036614,-0.029636,-0.029588,-0.017827,-0.029868,0.006927,...,0.002568,-0.045375,-0.008690,0.006083,0.011564,0.021049,-0.012042,-0.046045,-0.024235,-0.009452
2023-10-04,0.007309,0.001764,-0.002848,0.015740,0.018281,0.010893,0.001158,-0.001936,0.000000,-0.013759,...,-0.038745,0.029731,0.007215,0.032093,-0.047432,0.009214,0.010351,0.025412,0.009281,-0.002637
2023-10-05,0.007198,-0.001625,0.001052,0.005165,-0.008189,0.007259,0.004626,0.010912,0.017661,0.008878,...,-0.010326,0.004536,-0.052158,0.019378,-0.007702,-0.000464,0.000957,-0.009968,0.007706,0.011595
2023-10-06,0.014751,0.005358,-0.001301,0.020555,0.015878,0.015161,0.000384,0.011034,0.027908,-0.009790,...,0.033322,0.002258,0.001187,-0.003095,0.005415,0.005883,-0.001244,0.024612,-0.001477,0.005164
2023-10-09,0.008451,0.005869,0.006014,-0.006042,0.002344,-0.003348,0.009206,-0.005219,-0.002738,0.019449,...,0.045277,0.060887,0.006676,NaN,0.045601,0.014468,0.009865,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13,0.003969,-0.004325,0.008966,-0.033030,0.024843,-0.003015,0.000218,-0.004362,-0.016390,0.018317,...,0.016504,0.028950,0.002434,-0.003199,-0.016693,-0.002166,0.005330,0.058715,-0.001852,0.002936
2024-11-14,0.013770,-0.004227,-0.007234,0.009715,-0.012237,0.002294,0.000654,-0.000515,-0.027966,-0.006563,...,-0.008803,-0.002659,0.002489,-0.021550,-0.003212,-0.008455,-0.004800,-0.048201,-0.002204,0.013747
2024-11-15,-0.014109,-0.027354,-0.003514,0.003104,-0.041942,-0.004960,0.018518,0.009282,0.010549,0.019085,...,-0.013815,-0.047277,-0.039540,0.002343,-0.006214,0.013252,0.010992,-0.003650,-0.005347,-0.010547


In [35]:
def std_rank(returns):
    std_devs = returns.std()
    
    # Create a DataFrame with tickers and their respective standard deviations
    ranked_stocks = pd.DataFrame({
        "ticker": std_devs.index,
        "STD": std_devs.values
    })
    
    # Sort by standard deviation in ascending order (lowest STD at the top)
    ranked_stocks = ranked_stocks.sort_values(by="STD", ascending=True)
    
    return ranked_stocks['ticker'].values[:24].tolist()

ticker_lst = std_rank(tickers_raw_df)
print(ticker_lst)

['RY.TO', 'T.TO', 'PEP', 'TD.TO', 'SO', 'BK', 'COST', 'CSCO', 'IBM', 'AAPL', 'SPG', 'OXY', 'JPM', 'LOW', 'VZ', 'ABBV', 'CMCSA', 'AXP', 'BAC', 'GOOG', 'SLB', 'AMZN', 'GM', 'ORCL']


In [36]:

index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start="2024-11-20", end="2024-11-21")["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker):
    listed_currency = ticker.fast_info["currency"]
    if listed_currency == "CAD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
    elif listed_currency == "USD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"] * (1/exch_rate)
    raise ValueError("Currency not in CAD or USD")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    check_month_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        check_month_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker)
    ret_dataframe.drop(days_to_drop(check_month_dataframe), inplace=True)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst)
# display(stock_values)

[*********************100%***********************]  1 of 1 completed


In [37]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)
display(stock_returns)



,XIU.TO,^GSPC,Market Returns,RY.TO,T.TO,PEP,TD.TO,SO,BK,COST,...,VZ,ABBV,CMCSA,AXP,BAC,GOOG,SLB,AMZN,GM,ORCL
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03 00:00:00-04:00,-0.008913,-0.013744,-0.011329,-0.024235,0.006083,-0.008690,-0.009452,0.021049,-0.017827,-0.010975,...,0.006927,-0.005531,-0.016968,-0.029636,-0.029588,-0.013834,0.011564,-0.036614,-0.033570,-0.020523
2023-10-04 00:00:00-04:00,0.001384,0.008110,0.004747,0.009281,0.032093,0.007215,-0.002637,0.009214,-0.001936,0.011964,...,-0.013759,0.001764,0.000460,0.010893,0.001158,0.022281,-0.047432,0.018281,-0.010835,0.024493
2023-10-05 00:00:00-04:00,0.005872,-0.001304,0.002284,0.007706,0.019378,-0.052158,0.011595,-0.000464,0.010912,-0.003987,...,0.008878,-0.001625,-0.010352,0.007259,0.004626,-0.002055,-0.007702,-0.008189,-0.023518,0.011860
2023-10-06 00:00:00-04:00,0.004464,0.011815,0.008140,-0.001477,-0.003095,0.001187,0.005164,0.005883,0.011034,-0.021053,...,-0.009790,0.005358,0.010460,0.015161,0.000384,0.020148,0.005415,0.015878,0.019465,0.014859
2023-10-10 00:00:00-04:00,0.014359,0.011545,0.012952,0.005309,0.005322,0.025641,0.009022,0.022626,0.003559,0.008179,...,0.029498,0.004385,0.023004,0.010782,0.036057,0.003388,0.052424,0.011879,0.018770,-0.002274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25 00:00:00-04:00,-0.003760,-0.000299,-0.002030,-0.005724,-0.003623,-0.002149,-0.005714,-0.018481,-0.016994,-0.002462,...,-0.011467,-0.009491,0.000240,-0.009741,-0.017820,0.014952,0.011879,0.007780,-0.012329,-0.004932
2024-10-28 00:00:00-04:00,0.005392,0.002651,0.004022,0.012396,0.020455,-0.011351,-0.002043,-0.000979,0.017420,-0.000292,...,0.005800,0.009742,0.004321,0.015410,0.017427,0.008084,-0.008385,0.002981,0.012675,-0.006339
2024-10-29 00:00:00-04:00,-0.001341,0.001614,0.000137,-0.001393,-0.005791,-0.013778,-0.010494,-0.021462,0.000653,-0.004456,...,-0.006968,-0.001213,0.005497,-0.003831,-0.002112,0.016633,-0.023194,0.012952,-0.022568,0.005858


In [38]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


,RY.TO,T.TO,PEP,TD.TO,SO,BK,COST,CSCO,IBM,AAPL,SPG,OXY,JPM,LOW,VZ,ABBV,CMCSA,AXP,BAC,GOOG,SLB,AMZN,GM,ORCL
RY.TO,1.000000,0.360403,0.107992,0.345651,0.198141,0.499718,0.214936,0.260644,0.201870,0.169834,0.421949,0.145855,0.481190,0.375038,0.196441,0.146356,0.278749,0.467196,0.552770,0.110515,0.180349,0.215249,0.354763,0.097450
T.TO,0.360403,1.000000,0.191570,0.217608,0.438130,0.247272,0.040320,0.141166,0.159540,0.087701,0.307383,-0.022099,0.100290,0.259981,0.261789,0.169166,0.241562,0.108238,0.237257,0.068043,-0.011560,0.019007,0.133798,0.045721
PEP,0.107992,0.191570,1.000000,0.020676,0.394307,0.158149,0.124762,0.173425,0.136057,0.031303,0.070193,0.088710,0.036452,0.142870,0.232358,0.251739,0.237121,0.049386,0.129132,0.028023,0.008533,-0.074916,0.019791,0.019922
TD.TO,0.345651,0.217608,0.020676,1.000000,0.149081,0.361356,0.094742,0.151881,0.195040,0.067721,0.293448,0.246259,0.341685,0.305942,0.096051,0.081020,0.204840,0.373468,0.413507,0.074641,0.293213,0.084760,0.325526,0.152684
SO,0.198141,0.438130,0.394307,0.149081,1.000000,0.243349,0.016717,0.114603,0.123739,0.029753,0.244723,0.072757,0.125421,0.220622,0.322699,0.223209,0.225794,0.085016,0.271342,-0.003697,0.034990,-0.075742,0.129825,-0.015477
BK,0.499718,0.247272,0.158149,0.361356,0.243349,1.000000,0.213256,0.300136,0.341137,0.097776,0.434386,0.219488,0.482192,0.403395,0.189375,0.182658,0.277569,0.487607,0.575558,0.149427,0.255203,0.209161,0.405260,0.161985
COST,0.214936,0.040320,0.124762,0.094742,0.016717,0.213256,1.000000,0.180636,0.223431,0.261979,0.182514,0.019646,0.173805,0.162218,-0.076557,0.058528,0.028213,0.274116,0.163422,0.221063,0.039930,0.348700,0.091065,0.273913
CSCO,0.260644,0.141166,0.173425,0.151881,0.114603,0.300136,0.180636,1.000000,0.232615,0.156509,0.196706,0.204639,0.244709,0.283695,0.151227,0.143333,0.334626,0.250938,0.253531,0.221853,0.260599,0.282589,0.256240,0.161407
IBM,0.201870,0.159540,0.136057,0.195040,0.123739,0.341137,0.223431,0.232615,1.000000,0.113437,0.259745,0.073832,0.229521,0.229310,0.201966,0.138360,0.166246,0.237812,0.265773,0.076633,0.123606,0.156039,0.124855,0.225699
AAPL,0.169834,0.087701,0.031303,0.067721,0.029753,0.097776,0.261979,0.156509,0.113437,1.000000,0.184865,0.020879,0.056092,0.167125,-0.047188,-0.074323,0.154647,0.119738,0.120047,0.404677,0.083925,0.395391,0.074488,0.266004


In [39]:
#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)
print(stock_pairs)

[['COST', 'VZ'], ['SO', 'AMZN'], ['AAPL', 'ABBV'], ['T.TO', 'OXY'], ['CMCSA', 'ORCL'], ['PEP', 'SLB'], ['GOOG', 'GM'], ['TD.TO', 'CSCO'], ['RY.TO', 'IBM'], ['JPM', 'LOW'], ['SPG', 'AXP'], ['BK', 'BAC']]


In [40]:
#Arbitrarily assign a seed to keep results consistent
random.seed(21152764)

#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    print(len(stock_pairs))
    for i in range(6, len(stock_pairs)+1):
        print(i)
        #simple iteration, of 5 times
        for j in range(min(comb(len(stock_pairs), i), 5)):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

port_list = make_port_list(stock_pairs)

12
6
7
8
9
10
11
12


In [41]:
stock_values_just_date = stock_values
stock_values_just_date.index = pd.to_datetime(stock_values_just_date.index)
stock_values_just_date.index = stock_values_just_date.index.date

# Display the updated DataFrame
display(stock_values_just_date)


,XIU.TO,^GSPC,RY.TO,T.TO,PEP,TD.TO,SO,BK,COST,CSCO,...,VZ,ABBV,CMCSA,AXP,BAC,GOOG,SLB,AMZN,GM,ORCL
2023-10-02,28.043184,5984.447960,110.245735,20.027454,228.953331,75.579323,83.564845,56.477155,773.590134,72.981412,...,40.683823,197.608947,59.881305,205.287667,36.248025,188.167603,78.987112,180.661425,44.818887,146.578359
2023-10-03,27.793226,5902.197278,107.573982,20.149286,226.963824,74.864975,85.323822,55.470311,765.100190,72.508395,...,40.965639,196.515965,58.865215,199.203755,35.175515,185.564413,79.900497,174.046748,43.314335,143.570153
2023-10-04,27.831682,5950.062652,108.572334,20.795937,228.601454,74.667587,86.110037,55.362912,774.253643,72.238088,...,40.402012,196.862541,58.892311,201.373641,35.216247,189.698892,76.110640,177.228486,42.845031,147.086595
2023-10-05,27.995117,5942.303590,109.409035,21.198923,216.678081,75.533386,86.070058,55.967017,771.166407,71.724512,...,40.760682,196.542625,58.282663,202.835364,35.379157,189.309134,75.524442,175.777165,41.837397,148.831087
2023-10-06,28.120094,6012.511272,109.247398,21.133320,216.935202,75.923477,86.576431,56.584549,754.931285,72.238088,...,40.361645,197.595638,58.892311,205.910506,35.392731,193.123420,75.933418,178.568165,42.651783,151.042602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25,36.783108,8105.230979,170.220001,22.000000,239.732917,78.300003,127.168632,104.941587,1242.047028,77.785166,...,57.745787,262.144663,58.136525,373.086904,58.457489,233.034534,58.248168,262.116747,72.663679,242.147147
2024-10-28,36.981453,8126.721541,172.330002,22.450001,237.011696,78.139999,127.044075,106.769699,1241.684695,77.143232,...,58.080704,264.698409,58.387716,378.836357,59.476204,234.918446,57.759740,262.898223,73.584709,240.612088
2024-10-29,36.931870,8139.839103,172.089996,22.320000,233.746231,77.320000,124.317446,106.839468,1236.151927,77.701434,...,57.676013,264.377450,58.708680,377.385025,59.350609,238.825850,56.420061,266.303246,71.924066,242.021536
2024-10-30,36.872364,8112.975730,171.839996,22.100000,231.946046,77.389999,123.680777,105.960307,1225.142179,77.575839,...,57.606234,281.193228,58.945918,382.478591,59.043602,245.803350,56.029323,268.954688,72.524131,243.598457


In [42]:
portfolio_data = []

def portfolio_generator():
    global portfolio_data

    # creating portfolios -- starting wtih 12 stocks, going upto 24 stocks
    for i in range(len(port_list)):

        # for the ith portfolio, select the first i tickers
        selected_tickers = port_list[i]
        
        # objective function -- will be used in optimization steps
        # returns the tracking error between the portfolio's returns and the market's returns
        def objective(weights):
            return np.std(np.dot(stock_returns[selected_tickers].values, weights) - stock_returns['Market Returns'].values)
            # return np.std(np.dot(stock_returns[selected_tickers].mean(), weights) - stock_returns['Market Returns'].dropna().mean())
        
        # initial weights are calcualted so that, to start, every stock has equal weight
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # this constraint ensures that all of the stock weights sum to 1 (100%)
        def sum_weights(weights):
            return sum(weights) - 1
        
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # the bounds for each stock are that each stock must have a minimum weight of 1/2n% and maximum weight of 15%
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # optimize the portfolio weights
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        if result.success:

            weights = result.x
                        
            # calculate portfolio value and the number of shares to purchase
            portfolio_value = 1000000 * weights
            shares = portfolio_value / np.array([stock_values_just_date.at[TARGET_DATE, ticker] for ticker in selected_tickers]) 

            # calculate fees
            fee = np.minimum(3.95, 0.001 * shares)
            
            # subtract fees from portfolio value
            portfolio_value -= fee
            
            # Add to the results
            portfolio_data.append({
                'tickers': (selected_tickers),  # Tickers in the portfolio
                'weights': weights,  # Optimized weights
                'Tracking Error': result.fun,  # Standard deviation (risk)
                'Difference of Mean Returns': abs(np.dot(stock_returns[selected_tickers].values, weights).mean() - stock_returns['Market Returns'].values.mean()),
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_cost': portfolio_value.sum() + sum(fee),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_value.sum(),  # Total portfolio value after fees
                'final_fees': sum(fee)
            })

    return pd.DataFrame(portfolio_data)

portfolio_df = portfolio_generator()
# portfolio_df['weights'].iloc[0]

display(portfolio_df)

,tickers,weights,Tracking Error,Difference of Mean Returns,num_stocks,total_cost,shares_purchased,portfolio_value,final_fees
0,"[AAPL, ABBV, PEP, SLB, CMCSA, ORCL, TD.TO, CSC...","[0.12863531250836877, 0.06276765186082513, 0.0...",0.003669,0.000134,0,1.000000e+06,"[391.24935958144323, 238.28467916243162, 233.8...",999992.144024,7.855976
1,"[T.TO, OXY, SO, AMZN, PEP, SLB, SPG, AXP, RY.T...","[0.12233771129133655, 0.07421399497270105, 0.0...",0.003630,0.000134,1,1.000000e+06,"[5471.274953127699, 1020.7474264566431, 323.42...",999989.865798,10.134202
2,"[RY.TO, IBM, JPM, LOW, SPG, AXP, AAPL, ABBV, P...","[0.15, 0.06272483200004579, 0.0974124525666172...",0.003792,0.000211,2,1.000000e+06,"[874.9236697985524, 195.05777900473976, 311.46...",999994.634763,5.365237
3,"[SPG, AXP, GOOG, GM, BK, BAC, AAPL, ABBV, T.TO...","[0.041666666666666664, 0.08185736313731422, 0....",0.003852,0.000181,3,1.000000e+06,"[170.76281293352133, 215.3380871413765, 497.51...",999990.648716,9.351282
4,"[T.TO, OXY, GOOG, GM, RY.TO, IBM, CMCSA, ORCL,...","[0.12220876517247233, 0.09527718477581291, 0.1...",0.003738,0.000160,4,1.000000e+06,"[5465.508132226784, 1310.4528491657009, 571.27...",999989.627919,10.372081
5,"[JPM, LOW, RY.TO, IBM, BK, BAC, PEP, SLB, SPG,...","[0.050116243977327286, 0.06523485115765279, 0....",0.003343,0.000271,5,1.000000e+06,"[160.23895506485948, 172.45206615842778, 874.9...",999994.127741,5.872259
6,"[PEP, SLB, TD.TO, CSCO, RY.TO, IBM, SPG, AXP, ...","[0.03571428571428571, 0.045816044539243296, 0....",0.003911,0.000010,6,1.000000e+06,"[146.7710519655797, 783.0020885240821, 1172.96...",999989.905546,10.094454
7,"[GOOG, GM, CMCSA, ORCL, RY.TO, IBM, TD.TO, CSC...","[0.04251951124498226, 0.05985693228791041, 0.0...",0.003249,0.000206,7,1.000000e+06,"[182.64606563339387, 798.3030931669322, 615.94...",999992.930712,7.069288
8,"[CMCSA, ORCL, SO, AMZN, T.TO, OXY, GOOG, GM, R...","[0.03571428571428571, 0.05815026534027139, 0.0...",0.003266,0.000183,8,1.000000e+06,"[615.9438768835255, 237.69235348988337, 306.12...",999989.717755,10.282245
9,"[CMCSA, ORCL, JPM, LOW, SPG, AXP, PEP, SLB, AA...","[0.03571428571428571, 0.05928077201567388, 0.0...",0.003604,0.000276,9,1.000000e+06,"[615.9438768835255, 242.31336064677333, 293.05...",999993.698067,6.301933


When your code is run with our .csv file, it must create a DataFrame called “Portfolio_Final” where the index starts at 1 and ends at the number of stocks that your code chooses.  The headings must be as follows: Ticker, Price, Currency, Shares, Value, Weight. Ticker will be the ticker your code selected, Price is the price on November 25, 2023, Currency is either USD or CAD, Shares is the number of shares you purchased of that stock, Value is the total value of those shares, and Weight is the weight that the value of shares represents relative to the value of your portfolio (which again, must be $1,000,000 CAD).  You should show that your total adds to $1,000,000 CAD and that the weights add to 100%.  This DataFrame must be printed to the screen as the second to last output to the screen.  The final output to the screen will be a declaration to be detailed below.


In [43]:
temp = portfolio_df[portfolio_df['Difference of Mean Returns'] == portfolio_df['Difference of Mean Returns'].min()]


# Define columns for the final portfolio DataFrame
port = []

for i in range(len(temp['tickers'].iloc[0])):
    port.append({
        "Ticker": temp['tickers'].iloc[0][i],
        "Price": stock_values.loc[pd.to_datetime(TARGET_DATE).date(), temp['tickers'].iloc[0][i]],
        "Currency": yf.Ticker(temp['tickers'].iloc[0][i]).info['currency'],
        "Shares": temp['shares_purchased'].iloc[0][i],
        "Value": stock_values.loc[pd.to_datetime(TARGET_DATE).date(), temp['tickers'].iloc[0][i]] * temp['shares_purchased'].iloc[0][i],
        "Weight": temp['weights'].iloc[0][i]})

Portfolio_Final = pd.DataFrame(port)

# Reset index to start at 1
Portfolio_Final.index = Portfolio_Final.index + 1

Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight
1,JPM,312.759428,USD,228.490497,71462.557116,0.071463
2,LOW,378.278165,USD,123.726228,46802.930614,0.046803
3,CMCSA,57.983019,USD,538.950892,31250.000000,0.031250
4,ORCL,244.645082,USD,304.225912,74427.373156,0.074427
5,AAPL,328.780890,USD,456.230896,150000.000000,0.150000
6,ABBV,263.414551,USD,118.634297,31250.000000,0.031250
7,TD.TO,79.160004,CAD,1144.156490,90571.431977,0.090571
8,CSCO,78.985292,USD,706.740372,55822.094332,0.055822
9,BK,105.932402,USD,750.348929,79486.264131,0.079486
10,BAC,59.029644,USD,529.395025,31250.000000,0.031250


In [44]:
Portfolio_Final.to_csv("Stocks_Group_12.csv", sep='\t', encoding='UTF-8')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Alex, Derek & Sharuga